### Setup Conda Virtual Environment

IMPORTANT: Populate ```.env``` file in the home directory of your repository. Use the [.sample.env](.sample.env) as a reference guide.

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(), override=True)
print(os.getenv('AZURE_OPENAI_ENDPOINT'))

From the home directory run the folloing bash commands

```bash 
conda env create --name llmops --file=./conda.yaml
```  

```bash
conda activate llmops
```

### Create Local PromptFlow Connection to AOAI

In [ ]:
import subprocess
import os

api_key = os.getenv("AZURE_OPENAI_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
name = "aoai-connection"

command = f"pf connection create --file src/promptflows/connections/connect_aoai.yaml --set api_key={api_key} api_base={api_base} --name {name}"
result = subprocess.run(command,
                        shell=True,
                        check=True,
                        text=True,
                        capture_output=True)

print(result.stderr)

In [ ]:
!pf connection list

### Create a Remote Compute Target

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

In [ ]:
# Create Compute Target

# Name assigned to the compute cluster
cpu_compute_target = "remote-compute"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is.")

except Exception:
    print("Creating a new cpu compute target...")
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=1,
        idle_time_before_scale_down=300, # 5 minutes
        tier="Dedicated",
    )
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

print(cpu_cluster)